<a href="https://colab.research.google.com/github/stepan-ha/Labs/blob/main/Lab3K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Модель Блека-Шоулза. Метод Монте-Карло**

# **Інструкції**
Нехай маємо європейські та американські Call та Put опціони з такими параметрами:
* ціна відповідної акції в теперішній момент S=100$,
* ціна виконання опціону E=100$,
* час виконання опціону T = 1 рік,
* відсоткова ставка безризикового активу r=0.05,
* волатильність відповідної акції sigma=0.2.
Розробити програму, яка розраховує вартість європейських та американських Call та Put опціонів в теперішній момент часу t=0 за допомогою біноміального методу.
Для європейських Call та Put опціонів порівняти отримані результати з результатами, отриманими за формулами Блека-Шоулза.

# Імпорт бібліотек

---



In [1]:
import numpy as np
from scipy.stats import norm


# Ініціалізація Змінних

---



In [2]:

S = 100
E = 100
T = 1
r = 0.05
sigma = 0.2
N = 100

# Формула Блека-Шоулза

---



In [3]:
def black_scholes(S, E, T, r, sigma):
    d1 = (np.log(S / E) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call_price = S * norm.cdf(d1) - E * np.exp(-r * T) * norm.cdf(d2)
    put_price = E * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return call_price, put_price

# Біномінальний Метод




---



In [4]:
def binomial_option_pricing(S, E, T, r, sigma, N, option_type, european):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    ST = np.array([S * (u ** j) * (d ** (N - j)) for j in range(N + 1)])
    if option_type == "call":
        option_values = np.maximum(0, ST - E)
    else:
        option_values = np.maximum(0, E - ST)

    for i in range(N - 1, -1, -1):
        option_values = (p * option_values[1:i + 2] + (1 - p) * option_values[:i + 1]) * np.exp(-r * dt)

        if not european:
            ST = np.array([S * (u ** j) * (d ** (i - j)) for j in range(i + 1)])
            if option_type == "call":
                option_values = np.maximum(option_values, ST - E)
            else:
                option_values = np.maximum(option_values, E - ST)

    return option_values[0]

# Розв'язок

---



In [6]:
bs_call, bs_put = black_scholes(S, E, T, r, sigma)


binomial_euro_call = binomial_option_pricing(S, E, T, r, sigma, N, "call", european=True)
binomial_euro_put = binomial_option_pricing(S, E, T, r, sigma, N, "put", european=True)
binomial_call = binomial_option_pricing(S, E, T, r, sigma, N, "call", european=False)
binomial_put = binomial_option_pricing(S, E, T, r, sigma, N, "put", european=False)

print(f"Оптимальний Call: {bs_call}")
print(f"Оптимальний Put: {bs_put}")

print(f"Європейські Call: {binomial_euro_call}")
print(f"Європейські Put: {binomial_euro_put}")

print(f"Не Європейські Call: {binomial_call}")
print(f"Не Європейські Put: {binomial_put}")



Оптимальний Call: 10.450583572185565
Оптимальний Put: 5.573526022256971
Європейські Call: 10.430611662249113
Європейські Put: 5.553554112321353
Не Європейські Call: 10.430611662249113
Не Європейські Put: 6.082354409142444
